In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('maandamano_kenya_dataset.csv')

In [3]:
df.sample(3)

,Date,Location,Cause,Participants,Outcome,Organizer
443058,2017-02-01,Meru,Fuel Prices,94800,Violent Clashes,Union
681691,2020-11-27,Garissa,Unemployment,30339,Peaceful,Community Leaders
285808,2014-07-28,Nakuru,Climate Change,31478,Violent Clashes,Youth Movement


In [4]:
df.head()

,Date,Location,Cause,Participants,Outcome,Organizer
0,2010-01-01,Naivasha,Fuel Prices,9307,Arrests,Youth Movement
1,2010-01-01,Nakuru,Climate Change,7893,Arrests,Community Leaders
2,2010-01-01,Machakos,Climate Change,79809,Peaceful,Civil Society
3,2010-01-01,Eldoret,Land Grievances,97113,Arrests,Political Party
4,2010-01-01,Naivasha,Fuel Prices,66165,Violent Clashes,Civil Society


In [5]:
df.dtypes

Date            object
Location        object
Cause           object
Participants     int64
Outcome         object
Organizer       object
dtype: object

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col 

In [7]:
spark = SparkSession.builder \
        .master("local") \
        .appName("data cleaning with pyspark") \
        .getOrCreate()

25/01/22 11:40:56 WARN Utils: Your hostname, DESKTOP-EUV20F0 resolves to a loopback address: 127.0.1.1; using 172.30.123.101 instead (on interface eth0)
25/01/22 11:40:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/22 11:40:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
spark

25/01/22 11:41:19 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [9]:
df = spark.read.options(header='true', inferSchema='true').csv('maandamano_kenya_dataset.csv')

In [10]:
df.show()

+----------+--------+-------------------+------------+---------------+-----------------+
|      Date|Location|              Cause|Participants|        Outcome|        Organizer|
+----------+--------+-------------------+------------+---------------+-----------------+
|2010-01-01|Naivasha|        Fuel Prices|        9307|        Arrests|   Youth Movement|
|2010-01-01|  Nakuru|     Climate Change|        7893|        Arrests|Community Leaders|
|2010-01-01|Machakos|     Climate Change|       79809|       Peaceful|    Civil Society|
|2010-01-01| Eldoret|    Land Grievances|       97113|        Arrests|  Political Party|
|2010-01-01|Naivasha|        Fuel Prices|       66165|Violent Clashes|    Civil Society|
|2010-01-01|    Meru| Healthcare Strikes|       18066|       Peaceful|  Political Party|
|2010-01-01|  Kisumu|   Election Results|       41847|Violent Clashes|Community Leaders|
|2010-01-01|Naivasha|       Unemployment|        9610|       Peaceful|  Political Party|
|2010-01-01|Machakos|

In [11]:
df.schema

StructType([StructField('Date', DateType(), True), StructField('Location', StringType(), True), StructField('Cause', StringType(), True), StructField('Participants', IntegerType(), True), StructField('Outcome', StringType(), True), StructField('Organizer', StringType(), True)])

In [12]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Location: string (nullable = true)
 |-- Cause: string (nullable = true)
 |-- Participants: integer (nullable = true)
 |-- Outcome: string (nullable = true)
 |-- Organizer: string (nullable = true)



In [13]:
# Standardize column names to lowercase
df_cleaned = df.toDF(*[c.lower() for c in df.columns])

In [14]:
df_cleaned

DataFrame[date: date, location: string, cause: string, participants: int, outcome: string, organizer: string]

In [15]:
df_cleaned.show()

+----------+--------+-------------------+------------+---------------+-----------------+
|      date|location|              cause|participants|        outcome|        organizer|
+----------+--------+-------------------+------------+---------------+-----------------+
|2010-01-01|Naivasha|        Fuel Prices|        9307|        Arrests|   Youth Movement|
|2010-01-01|  Nakuru|     Climate Change|        7893|        Arrests|Community Leaders|
|2010-01-01|Machakos|     Climate Change|       79809|       Peaceful|    Civil Society|
|2010-01-01| Eldoret|    Land Grievances|       97113|        Arrests|  Political Party|
|2010-01-01|Naivasha|        Fuel Prices|       66165|Violent Clashes|    Civil Society|
|2010-01-01|    Meru| Healthcare Strikes|       18066|       Peaceful|  Political Party|
|2010-01-01|  Kisumu|   Election Results|       41847|Violent Clashes|Community Leaders|
|2010-01-01|Naivasha|       Unemployment|        9610|       Peaceful|  Political Party|
|2010-01-01|Machakos|